# How to explicitly define Schema

Schema inferencing doesnt work well for csv and json.
###### Lets create explicit schema for datframe and set them for reading csv and json file with correct schema

https://spark.apache.org/docs/latest/sql-ref-datatypes.html spark supported datatypes

Setting up schema is by two ways
1. programatically
2. Using DDL string - easy to use method
lets explore both method

In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession

from lib.logger import Log4j

spark = SparkSession.builder.master("local[3]").appName("Explicit Schema").getOrCreate()
logger = Log4j(spark)
logger.info("Starting HelloSparkSQL")

###### Define Schema for dataset progarmatically
Spark DataSchema is of StructType which is made up of list of fields.
structField take two mandatory arguments column name and datatype. 

StructType defines row structure and StructField is column definition.

In [2]:
from pyspark.sql.types import StructType, StructField, DateType, StringType, IntegerType
flightSchemaStruct = StructType([
        StructField("FL_DATE", DateType()),
        StructField("OP_CARRIER", StringType()),
        StructField("OP_CARRIER_FL_NUM", IntegerType()),
        StructField("ORIGIN", StringType()),
        StructField("ORIGIN_CITY_NAME", StringType()),
        StructField("DEST", StringType()),
        StructField("DEST_CITY_NAME", StringType()),
        StructField("CRS_DEP_TIME", IntegerType()),
        StructField("DEP_TIME", IntegerType()),
        StructField("WHEELS_ON", IntegerType()),
        StructField("TAXI_IN", IntegerType()),
        StructField("CRS_ARR_TIME", IntegerType()),
        StructField("ARR_TIME", IntegerType()),
        StructField("CANCELLED", IntegerType()),
        StructField("DISTANCE", IntegerType())
    ])

if datatype is not matched with data then spark throws an error at runtime. to handle error we must need to set mode.

In [3]:
flightTimeCsvDf = spark.read \
                    .format("csv") \
                    .option("header", "true") \
                    .option("mode", "FAILFAST") \
                    .schema(flightSchemaStruct) \
                    .load("data/flight*.csv")

In [4]:
flightTimeCsvDf.show(5)
logger.info("CSV Schema:" + flightTimeCsvDf.schema.simpleString())

Py4JJavaError: An error occurred while calling o72.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0, CT-LP-168, executor driver): org.apache.spark.SparkException: Malformed records are detected in record parsing. Parse Mode: FAILFAST. To process malformed records as null result, try setting the option 'mode' as 'PERMISSIVE'.
	at org.apache.spark.sql.catalyst.util.FailureSafeParser.parse(FailureSafeParser.scala:70)
	at org.apache.spark.sql.catalyst.csv.UnivocityParser$.$anonfun$parseIterator$2(UnivocityParser.scala:395)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:484)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:490)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:93)
	at org.apache.spark.sql.execution.datasources.

got an date time parse exception. Spark datatype not able to parse date field.

to handle this 

In [5]:
flightTimeCsvDf = spark.read \
                    .format("csv") \
                    .option("header", "true") \
                    .option("mode", "FAILFAST") \
                    .option("dateFormat", "M/d/y") \
                    .schema(flightSchemaStruct) \
                    .load("data/flight*.csv")

In [6]:
flightTimeCsvDf.show(5)
logger.info("CSV Schema:" + flightTimeCsvDf.schema.simpleString())

+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|ORIGIN_CITY_NAME|DEST|DEST_CITY_NAME|CRS_DEP_TIME|DEP_TIME|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|CANCELLED|DISTANCE|
+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|2000-01-01|        DL|             1451|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1115|    1113|     1343|      5|        1400|    1348|        0|     946|
|2000-01-01|        DL|             1479|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1315|    1311|     1536|      7|        1559|    1543|        0|     946|
|2000-01-01|        DL|             1857|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1415|    1414|     1642|      9|        1721|    1651|        0|     946

Bingo!!!  we did it.

###### Lets use DDL String for same

In [7]:
flightSchemaDDL = """FL_DATE DATE, OP_CARRIER STRING, OP_CARRIER_FL_NUM INT, ORIGIN STRING, 
          ORIGIN_CITY_NAME STRING, DEST STRING, DEST_CITY_NAME STRING, CRS_DEP_TIME INT, DEP_TIME INT, 
          WHEELS_ON INT, TAXI_IN INT, CRS_ARR_TIME INT, ARR_TIME INT, CANCELLED INT, DISTANCE INT"""

In [8]:
flightTimeCsvDf = spark.read \
                    .format("csv") \
                    .option("header", "true") \
                    .option("mode", "FAILFAST") \
                    .option("dateFormat", "M/d/y") \
                    .schema(flightSchemaDDL) \
                    .load("data/flight*.csv")

In [9]:
flightTimeCsvDf.show(5)
logger.info("CSV Schema:" + flightTimeCsvDf.schema.simpleString())

+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|ORIGIN_CITY_NAME|DEST|DEST_CITY_NAME|CRS_DEP_TIME|DEP_TIME|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|CANCELLED|DISTANCE|
+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|2000-01-01|        DL|             1451|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1115|    1113|     1343|      5|        1400|    1348|        0|     946|
|2000-01-01|        DL|             1479|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1315|    1311|     1536|      7|        1559|    1543|        0|     946|
|2000-01-01|        DL|             1857|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1415|    1414|     1642|      9|        1721|    1651|        0|     946

###### Repeat it for Json file

In [10]:
flightTimeJsonDf = spark.read \
                    .format("json") \
                    .option("mode", "FAILFAST") \
                    .option("dateFormat", "M/d/y") \
                    .schema(flightSchemaStruct) \
                    .load("data/flight*.json")
flightTimeJsonDf.show(5)
logger.info("JSON Schema:" + flightTimeJsonDf.schema.simpleString())

+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|ORIGIN_CITY_NAME|DEST|DEST_CITY_NAME|CRS_DEP_TIME|DEP_TIME|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|CANCELLED|DISTANCE|
+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|2000-01-01|        DL|             1451|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1115|    1113|     1343|      5|        1400|    1348|        0|     946|
|2000-01-01|        DL|             1479|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1315|    1311|     1536|      7|        1559|    1543|        0|     946|
|2000-01-01|        DL|             1857|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1415|    1414|     1642|      9|        1721|    1651|        0|     946

In [11]:
flightTimeJsonDf = spark.read \
                    .format("json") \
                    .option("mode", "FAILFAST") \
                    .option("dateFormat", "M/d/y") \
                    .schema(flightSchemaDDL) \
                    .load("data/flight*.json")
flightTimeJsonDf.show(5)
logger.info("JSON Schema:" + flightTimeJsonDf.schema.simpleString())

+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|ORIGIN_CITY_NAME|DEST|DEST_CITY_NAME|CRS_DEP_TIME|DEP_TIME|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|CANCELLED|DISTANCE|
+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|2000-01-01|        DL|             1451|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1115|    1113|     1343|      5|        1400|    1348|        0|     946|
|2000-01-01|        DL|             1479|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1315|    1311|     1536|      7|        1559|    1543|        0|     946|
|2000-01-01|        DL|             1857|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1415|    1414|     1642|      9|        1721|    1651|        0|     946

In [12]:
spark.stop()